# TP 2

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import matplotlib
import plotly as py
import plotly.express as px 
import matplotlib.ticker as ticker
plt.style.use('default')
import plotly.offline as opy
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
from datetime import datetime

sns.set(style="whitegrid")

pd.options.display.float_format = '{:20,.2f}'.format 

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Ejecutar si no funciona el import
!pip3 install catboost

     |████████████████████████████████| 65.8MB 67kB/s 


In [3]:
from catboost import CatBoostClassifier

## lectura de datos

In [4]:
entrenamiento = pd.read_csv('Training.csv')
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  16947 non-null  int64  
 1   Region                              16947 non-null  object 
 2   Territory                           16947 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 5   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 6   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 7   Submitted_for_Approval              16947 non-null  int64  
 8   Bureaucratic_Code                   16947 non-null  object 
 9   Account_Created_Date                16947 non-null  object 
 10  Source                              16947 non-null  object 
 11  Billing_Country                     16947

In [5]:
testeo = pd.read_csv('Test.csv')
testeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  2551 non-null   int64  
 1   Region                              2551 non-null   object 
 2   Territory                           2551 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 7   Submitted_for_Approval              2551 non-null   int64  
 8   Bureaucratic_Code                   2551 non-null   object 
 9   Account_Created_Date                2551 non-null   object 
 10  Source                              2551 non-null   object 
 11  Billing_Country                     2551 no

## prosesamiento de los df

### modificaciones de oportunidad

In [6]:
def convertirATiempo(df,lista):
    for columna in lista:
        df[columna] = pd.to_datetime(df[columna])
    return df

def convertirATipo(df,lista,tipo):
    for columna in lista:
        df[columna] = df[columna].astype(tipo)
    return df

def convertirAFlotante(df,lista):
    for columna in lista:
        df[columna] = df[columna].apply(lambda x: None if (x == "None" or x == "Other") else float(x))
    return df

def establecerTipos(df):
    """cambia los tipos de las columnas"""
    
    #fechas
    listaDeColumnasFechas = ['Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Last_Modified_Date','Delivery_Year','Planned_Delivery_End_Date','Planned_Delivery_Start_Date']
    df = convertirATiempo(df,listaDeColumnasFechas)
    #enteros
    listaDeColumnasEnteras = ['ID','Opportunity_ID']
    convertirATipo(df,listaDeColumnasEnteras,'int')
    #decimales
    listaDeColumnasEnteras = ['Price','ASP','ASP_(converted)','Total_Taxable_Amount','Total_Amount']
    convertirAFlotante(df,listaDeColumnasEnteras)
    #booleanos
    listaDeColumnasBooleanas = ['Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved','Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved']
    convertirATipo(df,listaDeColumnasBooleanas,'bool')
    #categoricas
    listaDeColumnasCategoricas = ['Product_Category_B','Product_Family','Region','Territory','Bureaucratic_Code','Source ','Billing_Country','Account_Name','Opportunity_Name']
    convertirATipo(df,listaDeColumnasCategoricas,'category')
    return df

def eliminarColumnasSinImportancia(df):
    #"Last_Activity" y "Actual_Delivery_Date" son puros nulos
    #"Submitted_for_Approval" puros valores nulos
    #"Prod_Category_A" tiene todos sus valores iguales
    df = df.drop(['Submitted_for_Approval','Last_Activity','Actual_Delivery_Date','Prod_Category_A'], axis=1)
    return df

def agregarColumnas(df):
    df['Closed'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': True, 'Negotiation':False,'Proposal':False,'Qualification':False})#REVISAR
    df['Won'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': False, 'Negotiation':False,'Proposal':False,'Qualification':False})
    convertirATipo(df,['Stage'],'category')
    return df

def correccionDeTerritorios(df):
    #se refiere a diferentes partes de usa
    df = df.replace({'Territory':{'Central US':'USA','NE America':'USA', 'NW America':'USA','SW America':'USA','SE America':'USA'}})
    #
    df = df.replace({'Territory':{'KSA':'Saudi Arabia'}})
    #No se ha encontrado la dorma de los siguientes datos 'Kosovo','South America', 'South East Asia', 'UAE (Dubai)', 'UAE (Others)' (en el caso de kosovo no lo grafica)
    return df

def preparacionDelDataFrame(df,entrenamiento=True):
    df = eliminarColumnasSinImportancia(df)
    df = establecerTipos(df)
    df = correccionDeTerritorios(df)
    df.rename(columns = {'Source ':'Source'}, inplace = True)
    if (entrenamiento):
      df = agregarColumnas(df)
    return df

### por oportunidad

In [22]:
def desacerseDeNulosDateTime(df,columna):
  df[columna] = df[columna].apply(lambda x: datetime.max if (x is None or x is np.NaN or x is pd.NaT) else x)
  return df

def descomposicionDeFechas(df,lista):
  """se nesesitan enteros no fechas"""
  for columna in lista:
    anio = columna + '_anio'
    df[anio] = df[columna].apply(lambda x: x if x is None else x.year)
    mes = columna + '_mes'
    df[mes] = df[columna].apply(lambda x: x if x is None else x.month)
    dia = columna + '_dia'
    df[dia] = df[columna].apply(lambda x: x if x is None else x.day)
  df = df.drop(lista, axis=1)
  return df

In [74]:
def dataEntrenamiento(df):
  data = df.groupby('Opportunity_ID').agg({'Won':'sum','Closed':'sum','modificaciones':'sum',
                                           #'Won':'sum','modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           #'Price':'mean',#Demaciados nulos
                                           'Quote_Expiry_Date':'max',
                                           'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           'Pricing, Delivery_Terms_Approved':'sum',
                                           'Bureaucratic_Code_0_Approval':'sum',
                                           'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Account_Type':'first',
                                           'Delivery_Quarter':'first',
                                           'Product_Type':'first',
                                           'Product_Category_B':'first',
                                           'Billing_Country':'first'
                                           })#hay que seguir agregando cosas que son importantes
  return data

def dataTesteo(df):
  data = df.groupby('Opportunity_ID').agg({'modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           #'Price':'mean',#Demaciados nulos
                                           'Quote_Expiry_Date':'max',
                                           'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           'Pricing, Delivery_Terms_Approved':'sum',
                                           'Bureaucratic_Code_0_Approval':'sum',
                                           'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Account_Type':'first',
                                           'Delivery_Quarter':'first',
                                           'Product_Type':'first',
                                           'Product_Category_B':'first',
                                           'Billing_Country':'first'
                                           })#hay que seguir agregando cosas que son importantes
  return data  

def agrupacionDeOportunidades(df,entrenamiento=True):
  df['modificaciones'] = 1
  if (entrenamiento):
    data = dataEntrenamiento(df)
    #Solo me interesa saber si se cerro con exito
    #data['Closed'] = data['Closed'].agg(lambda x: 1 if x > 0 else 0)
    data['Won'] = data['Won'].agg(lambda x: 1 if x > 0 else 0)
    #data['Lost'] = data['Closed'] - data['Won']#si es won es closed pero si es closed y no es won es perdido
    #data['Lost'] = data['Lost'].agg(lambda x: 1 if x > 0 else 0)

    #REVISAR la importancia del caso no cerrado
    data = data[data['Closed'] > 0]#no interesan los que no se cerraron, dado que no se cerraron no aportan
    data = data.drop(['Closed'], axis=1)
  else:
    data = dataTesteo(df)

  data['Pricing, Delivery_Terms_Quote_Appr'] = data['Pricing, Delivery_Terms_Quote_Appr'].agg(lambda x: 1 if x > 0 else 0)
  data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Approved'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approval'] = data['Bureaucratic_Code_0_Approval'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approved'].agg(lambda x: 1 if x > 0 else 0)
  #data = data.replace({'Won':})

  #categoricas
  #listaDeColumnasCategoricas = ['Account_Type','Region','Territory','Delivery_Quarter']
  listaDeColumnasCategoricas = ['Account_Type','Region','Territory','Billing_Country','Delivery_Quarter']
  convertirATipo(data,listaDeColumnasCategoricas,'category')

  #data = desacerseDeNulosDateTime(data,'Quote_Expiry_Date')#si no hay fecha de expiracion se pre supone que es infinita

  listaDeValoresDeFechas = ['Last_Modified_Date','Opportunity_Created_Date','Account_Created_Date','Quote_Expiry_Date']
  data = descomposicionDeFechas(data,listaDeValoresDeFechas)

  return data

las siguientes columnas fueron ignoradas hay que analizarlas

Posibles columnas a usar:

- Billing_Country (posibles datos repetidos)
- Bureaucratic_Code
- Delivery_Terms
- Delivery_Year (posibles datos repetidos)
- Month (posibles datos repetidos)
- Opportunity_Owner
- Opportunity_Type
- Product_Family
- Product_Name
- Quote_Type (muy desbalanceado)
- TRF (muy desbalanceado)

Revisar el contexto

- Total_Amount_Currency
- Total_Taxable_Amount
- Total_Taxable_Amount_Currency
- ASP
- ASP_(converted)_Currency
- ASP_Currency

Tal vez usar en conjunto

- Planned_Delivery_End_Date y Planned_Delivery_Start_Date (diferencia)

revisar los siguentes:

- Product_Type: Excesiva cantidad de nones
- Product_Category_B: Excesiva cantidad de nones

las columnas con gran catidad de nones parecen afectar positivamente el resultado hay que revisarlas una por una

- Currency: Excesiva cantidad de nones
- Brand: Excesiva cantidad de nones
- Sales_Contract_No: Excesiva cantidad de nones
- Size: Excesiva cantidad de nones
- Source: Excesiva cantidad de nones

In [86]:
agrupacionEntrenamiento['Delivery_Quarter'].value_counts()

Q4    2570
Q3    2439
Q2    2427
Q1    2355
Name: Delivery_Quarter, dtype: int64

## Ejecucion de cambios

### ejecucion por modificaciones de oportunidad

In [10]:
entrenamientoProcesado = preparacionDelDataFrame(entrenamiento)
entrenamientoProcesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  16947 non-null  int64         
 1   Region                              16947 non-null  category      
 2   Territory                           16947 non-null  object        
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  bool          
 4   Pricing, Delivery_Terms_Approved    16947 non-null  bool          
 5   Bureaucratic_Code_0_Approval        16947 non-null  bool          
 6   Bureaucratic_Code_0_Approved        16947 non-null  bool          
 7   Bureaucratic_Code                   16947 non-null  category      
 8   Account_Created_Date                16947 non-null  datetime64[ns]
 9   Source                              16947 non-null  category      
 10  Billing_Country       

In [11]:
testeoProsesado = preparacionDelDataFrame(testeo,False)
testeoProsesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 47 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  2551 non-null   int64         
 1   Region                              2551 non-null   category      
 2   Territory                           2551 non-null   object        
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   bool          
 4   Pricing, Delivery_Terms_Approved    2551 non-null   bool          
 5   Bureaucratic_Code_0_Approval        2551 non-null   bool          
 6   Bureaucratic_Code_0_Approved        2551 non-null   bool          
 7   Bureaucratic_Code                   2551 non-null   category      
 8   Account_Created_Date                2551 non-null   datetime64[ns]
 9   Source                              2551 non-null   category      
 10  Billing_Country         

### ejecucion por oportunidad

In [75]:
agrupacionEntrenamiento = agrupacionDeOportunidades(entrenamientoProcesado)
agrupacionEntrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   Won                                 9791 non-null   int64   
 1   modificaciones                      9791 non-null   int64   
 2   Pricing, Delivery_Terms_Quote_Appr  9791 non-null   int64   
 3   Pricing, Delivery_Terms_Approved    9791 non-null   int64   
 4   Bureaucratic_Code_0_Approval        9791 non-null   int64   
 5   Bureaucratic_Code_0_Approved        9791 non-null   int64   
 6   Total_Amount                        9791 non-null   float64 
 7   Region                              9791 non-null   category
 8   Territory                           9791 non-null   category
 9   ASP_(converted)                     9791 non-null   float64 
 10  Account_Type                        9791 non-null   category
 11  Delivery_Quarter             

In [76]:
agrupacionTesteo = agrupacionDeOportunidades(testeoProsesado,False)
agrupacionTesteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   modificaciones                      1567 non-null   int64   
 1   Pricing, Delivery_Terms_Quote_Appr  1567 non-null   int64   
 2   Pricing, Delivery_Terms_Approved    1567 non-null   int64   
 3   Bureaucratic_Code_0_Approval        1567 non-null   int64   
 4   Bureaucratic_Code_0_Approved        1567 non-null   int64   
 5   Total_Amount                        1567 non-null   float64 
 6   Region                              1567 non-null   category
 7   Territory                           1567 non-null   category
 8   ASP_(converted)                     1567 non-null   float64 
 9   Account_Type                        1567 non-null   category
 10  Delivery_Quarter                    1567 non-null   category
 11  Product_Type             

### separacion de resultados de entrenamiento

In [77]:
entrenamientoResultado = agrupacionEntrenamiento[['Won']]
entrenamientoResultado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Won     9791 non-null   int64
dtypes: int64(1)
memory usage: 153.0 KB


In [78]:
agrupacionEntrenamiento = agrupacionEntrenamiento.drop(['Won'], axis=1)
agrupacionEntrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9791 entries, 0 to 12803
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   modificaciones                      9791 non-null   int64   
 1   Pricing, Delivery_Terms_Quote_Appr  9791 non-null   int64   
 2   Pricing, Delivery_Terms_Approved    9791 non-null   int64   
 3   Bureaucratic_Code_0_Approval        9791 non-null   int64   
 4   Bureaucratic_Code_0_Approved        9791 non-null   int64   
 5   Total_Amount                        9791 non-null   float64 
 6   Region                              9791 non-null   category
 7   Territory                           9791 non-null   category
 8   ASP_(converted)                     9791 non-null   float64 
 9   Account_Type                        9791 non-null   category
 10  Delivery_Quarter                    9791 non-null   category
 11  Product_Type                 

## Prueva

In [ ]:
prueva = agrupacionTesteo.reset_index()
prueva = prueva[['Opportunity_ID']]
prueva['prediccion'] = 0.5
prueva
prueva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Opportunity_ID  1567 non-null   int64
 1   prediccion      1567 non-null   int64
dtypes: int64(2)
memory usage: 24.6 KB


In [ ]:
prueva = agrupacionTesteo
prueva['Target'] = 0.5
prueva = prueva[['Target']]
prueva.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Target  1567 non-null   float64
dtypes: float64(1)
memory usage: 24.5 KB


In [ ]:
from google.colab import files

prueva.to_csv('prueva.csv')
#files.download('resultado.csv')

## Resultado

In [79]:
def generarResultado(df,prediccion):
  resultado = df
  resultado['Target'] = prediccion
  resultado = resultado[['Target']]
  resultado.to_csv('resultado.csv')
  return resultado

In [80]:
def auxiliarUnificarValor(arreglo):
  return arreglo[:,0]

## Catboost

In [81]:
def catboost(train_data,train_labels,cat_features,eval_data):
  # Inicializador de CatBoostClassifier
  model = CatBoostClassifier(iterations=10,
                             learning_rate=0.5,
                             depth=8,
                             l2_leaf_reg=3.5,
                             rsm=0.7
                             #use_best_model=True#Requiere data de evalucion
                             )
  #parametros importantes por agregar: l2_leaf_reg, rsm
  #para tipos de category: cat_features, one_hot_max_size

  # Entrenamiento del modelo
  model.fit(train_data, train_labels, cat_features)

  # obtener prediccion
  #preds_class = model.predict(eval_data)

  # obtener probabilidad de prediccion
  preds_proba = model.predict_proba(eval_data)

  # obtener RawFormulaVal de la prediccion
  #preds_raw = model.predict(eval_data, prediction_type='RawFormulaVal')

  #return resultado(eval_data,preds_proba)
  return preds_proba

In [82]:
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B']
#listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B','Account_Type','Delivery_Quarter']
listaDeCategoricas = ['Region','Territory','Product_Type','Product_Category_B','Account_Type','Delivery_Quarter','Billing_Country']
#listaDeCategoricas = []
res = catboost(agrupacionEntrenamiento,entrenamientoResultado,listaDeCategoricas,agrupacionTesteo)
res

0:	learn: 0.5477704	total: 17ms	remaining: 153ms
1:	learn: 0.4769192	total: 32.8ms	remaining: 131ms
2:	learn: 0.4408569	total: 46.7ms	remaining: 109ms
3:	learn: 0.4192824	total: 60.2ms	remaining: 90.3ms
4:	learn: 0.4158760	total: 71.2ms	remaining: 71.2ms
5:	learn: 0.3923950	total: 87.2ms	remaining: 58.1ms
6:	learn: 0.3849204	total: 101ms	remaining: 43.1ms
7:	learn: 0.3783924	total: 116ms	remaining: 29ms
8:	learn: 0.3670690	total: 131ms	remaining: 14.5ms
9:	learn: 0.3614815	total: 145ms	remaining: 0us


array([[0.18759446, 0.81240554],
       [0.63254171, 0.36745829],
       [0.22517343, 0.77482657],
       ...,
       [0.43087198, 0.56912802],
       [0.46809216, 0.53190784],
       [0.84376153, 0.15623847]])

In [83]:
resultado = auxiliarUnificarValor(res)
resultado = (resultado - 1) * (-1)#En las ultimas pruevas es el opuesto el que genera mejor resultado
resultado

array([0.81240554, 0.36745829, 0.77482657, ..., 0.56912802, 0.53190784,
       0.15623847])

In [84]:
dataResultante = generarResultado(agrupacionTesteo,resultado)
dataResultante.head()

,Target
Opportunity_ID,
10689,0.81
10690,0.37
10691,0.77
10692,0.73
10693,0.69
